In [1]:
from pymatgen import MPRester, Composition
from pymatgen.electronic_structure.plotter import BSPlotter
import pymatgen.analysis.find_dimension
import pymatgen.io.cif as pcif
import pandas as pd
import pymatgen.analysis.find_dimension as pfd
from tqdm import tqdm

supported_properties = ("energy", "energy_per_atom", "volume",
                            "formation_energy_per_atom", "nsites",
                            "unit_cell_formula", "pretty_formula",
                            "is_hubbard", "elements", "nelements",
                            "e_above_hull", "hubbards", "is_compatible",
                            "spacegroup", "task_ids", "band_gap", "density",
                            "icsd_id", "icsd_ids", "cif", "total_magnetization",
                            "material_id", "oxide_type", "tags", "elasticity")

supported_task_properties = ("energy", "energy_per_atom", "volume",
                                 "formation_energy_per_atom", "nsites",
                                 "unit_cell_formula", "pretty_formula",
                                 "is_hubbard",
                                 "elements", "nelements", "e_above_hull",
                                 "hubbards",
                                 "is_compatible", "spacegroup",
                                 "band_gap", "density", "icsd_id", "cif")
quasi_1d = ("TaSe3","NbS3","NbSe3","TiS3","HfS3","HfSe3","ZrS3","ZrSe3","ZrTe3")

## 1. Connect to MPDB and download data

In [25]:
mpr = MPRester("Frv0akZ1InOToUmL")##API key should be inside the "", URL for API key: https://materialsproject.org/dashboard
data = mpr.query(criteria={"band_gap": {"$gt": -0.1}},properties=(["cif","pretty_formula","unit_cell_formula","band_gap",\
                                                                   "volume","spacegroup","density","elements","nelements"]))
df_all = pd.DataFrame(data)

D:\ReserachTool\Anaconda\lib\site-packages\ipykernel_launcher.py:1: ResourceWarning: unclosed <socket.socket fd=3000, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.13.163.62', 53048), raddr=('104.25.154.118', 443)>
  """Entry point for launching an IPython kernel.


In [26]:
df_all.head()

,cif,pretty_formula,unit_cell_formula,band_gap,volume,spacegroup,density,elements,nelements
0,# generated using pymatgen\ndata_Mo3P\n_symmet...,Mo3P,"{'P': 4.0, 'Mo': 12.0}",0.0,235.500240,"{'source': 'spglib', 'symbol': 'I-42m', 'numbe...",8.991405,"[Mo, P]",2
1,# generated using pymatgen\ndata_WS\n_symmetry...,WS,"{'W': 2.0, 'S': 2.0}",0.0,64.852032,"{'source': 'spglib', 'symbol': 'P6_3mc', 'numb...",11.056513,"[W, S]",2
2,# generated using pymatgen\ndata_HfAu\n_symmet...,HfAu,"{'Hf': 2.0, 'Au': 2.0}",0.0,78.612582,"{'source': 'spglib', 'symbol': 'P4/nmm', 'numb...",15.861590,"[Au, Hf]",2
3,# generated using pymatgen\ndata_SbIr\n_symmet...,SbIr,"{'Sb': 2.0, 'Ir': 2.0}",0.0,80.038446,"{'source': 'spglib', 'symbol': 'P6_3/mmc', 'nu...",13.028016,"[Sb, Ir]",2
4,# generated using pymatgen\ndata_Zn3(Ni10B3)2\...,Zn3(Ni10B3)2,"{'B': 6.0, 'Ni': 20.0, 'Zn': 3.0}",0.0,288.696098,"{'source': 'spglib', 'symbol': 'Fm-3m', 'numbe...",8.253692,"[B, Ni, Zn]",3


In [28]:
df_all.to_excel('Material_Project_dataForDimensionClassify.xlsx')

In [35]:
#for i in tqdm(range(len(data))):
#    if len(data[i]) < 1:
#        data[i] = "Nan"
#data = [e for e in data if e != "Nan"]




100%|█████████████████████████████████████████████████████████████████████| 120612/120612 [00:00<00:00, 2238481.53it/s]


## 2. Save structures into cif files

In [59]:
subsample = 100
for i in range(len(data[:subsample])):
    with open(str(i) + '_dimension.cif', 'w') as the_file:
        the_file.write(data[i]['cif'])

## 3. Dimension Classification

In [60]:
dimensions = []
for i in tqdm(range(len(data[:subsample]))):
    dim = pfd.find_dimension(pcif.Structure.from_file(str(i)+"_dimension.cif"))
    if dim == '2D' and  pfd.find_dimension(pcif.Structure.from_file(str(i)+"_dimension.cif"),tolerance=0.1) == "1D":
        dimensions.append('quasi-1D')
    else:
        dimensions.append(dim)





  0%|                                                                                          | 0/100 [00:00<?, ?it/s]D:\ReserachTool\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_dimension is deprecated
find_dimension has been moved topymatgen.analysis.dimensionality.get_dimensionality_cheon. This method will be removed in pymatgen v2019.1.1.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:156: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list1 = find_connected_atoms(structure, tolerance=tolerance, ldict=ldict)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:157: DeprecationWarning: find_clusters is deprecated
find_clusters has been moved topym

D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:156: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list1 = find_connected_atoms(structure, tolerance=tolerance, ldict=ldict)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:157: DeprecationWarning: find_clusters is deprecated
find_clusters has been moved topymatgen.analysis.dimensionality.find_clusters. This method will be removed in pymatgen v2019.1.1.
  max1, min1, clusters1 = find_clusters(structure, connected_list1)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:159: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list

D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:160: DeprecationWarning: find_clusters is deprecated
find_clusters has been moved topymatgen.analysis.dimensionality.find_clusters. This method will be removed in pymatgen v2019.1.1.
  max2, min2, clusters2 = find_clusters(structure, connected_list2)




  8%|██████▌                                                                           | 8/100 [02:28<26:25, 17.24s/it]D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\io\cif.py:1116: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
D:\ReserachTool\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_dimension is deprecated
find_dimension has been moved topymatgen.analysis.dimensionality.get_dimensionality_cheon. This method will be removed in pymatgen v





 12%|█████████▋                                                                       | 12/100 [02:31<06:56,  4.73s/it]D:\ReserachTool\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_dimension is deprecated
find_dimension has been moved topymatgen.analysis.dimensionality.get_dimensionality_cheon. This method will be removed in pymatgen v2019.1.1.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:156: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list1 = find_connected_atoms(structure, tolerance=tolerance, ldict=ldict)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:157: DeprecationWarning: find_clusters is deprecated
find_clusters has been moved topym

D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:156: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list1 = find_connected_atoms(structure, tolerance=tolerance, ldict=ldict)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:157: DeprecationWarning: find_clusters is deprecated
find_clusters has been moved topymatgen.analysis.dimensionality.find_clusters. This method will be removed in pymatgen v2019.1.1.
  max1, min1, clusters1 = find_clusters(structure, connected_list1)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:159: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list

...

D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:156: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list1 = find_connected_atoms(structure, tolerance=tolerance, ldict=ldict)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:157: DeprecationWarning: find_clusters is deprecated
find_clusters has been moved topymatgen.analysis.dimensionality.find_clusters. This method will be removed in pymatgen v2019.1.1.
  max1, min1, clusters1 = find_clusters(structure, connected_list1)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:159: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list

D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:160: DeprecationWarning: find_clusters is deprecated
find_clusters has been moved topymatgen.analysis.dimensionality.find_clusters. This method will be removed in pymatgen v2019.1.1.
  max2, min2, clusters2 = find_clusters(structure, connected_list2)




 73%|███████████████████████████████████████████████████████████▏                     | 73/100 [12:44<02:45,  6.14s/it]D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\io\cif.py:1116: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
D:\ReserachTool\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_dimension is deprecated
find_dimension has been moved topymatgen.analysis.dimensionality.get_dimensionality_cheon. This method will be removed in pymatgen v

D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:159: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list2 = find_connected_atoms(structure, tolerance=tolerance, ldict=ldict)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:160: DeprecationWarning: find_clusters is deprecated
find_clusters has been moved topymatgen.analysis.dimensionality.find_clusters. This method will be removed in pymatgen v2019.1.1.
  max2, min2, clusters2 = find_clusters(structure, connected_list2)




 81%|█████████████████████████████████████████████████████████████████▌               | 81/100 [16:50<06:35, 20.80s/it]D:\ReserachTool\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_dimension is deprecated
find_dimension has been moved topymatgen.analysis.dimensiona

D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:156: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list1 = find_connected_atoms(structure, tolerance=tolerance, ldict=ldict)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:157: DeprecationWarning: find_clusters is deprecated
find_clusters has been moved topymatgen.analysis.dimensionality.find_clusters. This method will be removed in pymatgen v2019.1.1.
  max1, min1, clusters1 = find_clusters(structure, connected_list1)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:159: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list

D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:159: DeprecationWarning: find_connected_atoms is deprecated
find_connected_atoms has been moved to pymatgen.analysis.dimensionality.find_connected_atoms. This method will be removed in pymatgen v2019.1.1.
  connected_list2 = find_connected_atoms(structure, tolerance=tolerance, ldict=ldict)
D:\ReserachTool\Anaconda\lib\site-packages\pymatgen\analysis\find_dimension.py:160: DeprecationWarning: find_clusters is deprecated
find_clusters has been moved topymatgen.analysis.dimensionality.find_clusters. This method will be removed in pymatgen v2019.1.1.
  max2, min2, clusters2 = find_clusters(structure, connected_list2)




 94%|████████████████████████████████████████████████████████████████████████████▏    | 94/100 [19:11<00:54,  9.09s/it]D:\ReserachTool\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_dimension is deprecated
find_dimension has been moved topymatgen.analysis.dimensiona

## 3. Save the file into excel format

In [61]:
pretty_formula = []
band_gap = []
for i in range(len(data[:subsample])):
    pretty_formula.append(data[i]['pretty_formula'])
    band_gap.append(data[i]['band_gap'])
df_dimensions = pd.DataFrame({'pretty_formula':pretty_formula,'band_gap':band_gap,'dimensions':dimensions})

In [63]:
df_dimensions.to_excel('dimResult.xlsx')